<h1>sign laungauge interpertation</h1>



<h3>Import libraryes</h3>

In [1]:
import tensorflow as tf
import cv2
import matplotlib as plt
import numpy as np
import time
import mediapipe as mp

<h3>Recording frames & getting Landmarks</h3>


In [2]:
mp_hol=mp.solutions.holistic
mp_draw=mp.solutions.drawing_utils

In [3]:
def draw(frame,landmarks,mp_draw,mp_hol):
        mp_draw.draw_landmarks(frame,landmarks.face_landmarks,mp_hol.FACEMESH_CONTOURS,
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1))
        mp_draw.draw_landmarks(frame,landmarks.left_hand_landmarks,mp_hol.HAND_CONNECTIONS,
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1))
        mp_draw.draw_landmarks(frame,landmarks.right_hand_landmarks,mp_hol.HAND_CONNECTIONS,
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1))
        mp_draw.draw_landmarks(frame,landmarks.pose_landmarks,mp_hol.POSE_CONNECTIONS,
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1))

In [4]:
#just capture
cam=cv2.VideoCapture(0)

with mp_hol.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hol:
    while cam.isOpened():
        ret,frame=cam.read()
        if not ret:
            break
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        landmarks=hol.process(image)
           
        #drawing on image
        draw(frame,landmarks,mp_draw,mp_hol)
        
        frame=cv2.flip(frame,1)
        cv2.imshow("capture",frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cam.release()
    cv2.destroyAllWindows()


<h3>Extracting landmarks</h3>

In [5]:
def extract_landmarks(landmarks):
    if landmarks.pose_landmarks:
        pose=np.array([[p.x,p.y,p.z,p.visibility] for p in landmarks.pose_landmarks.landmark]).flatten()
    else:
        pose=np.zeros(132,)
    if landmarks.left_hand_landmarks:
        left_hand=np.array([[p.x,p.y,p.z] for p in landmarks.left_hand_landmarks.landmark]).flatten()
    else:
        left_hand=np.zeros(63,)
    if landmarks.right_hand_landmarks:
        right_hand=np.array([[p.x,p.y,p.z] for p in landmarks.right_hand_landmarks.landmark]).flatten()
    else:
        right_hand=np.zeros(63,)
    if landmarks.face_landmarks:
        face=np.array([[p.x,p.y,p.z] for p in landmarks.face_landmarks.landmark]).flatten()
    else:
        face=np.zeros(1404,)
    return np.concatenate([face,pose,left_hand,right_hand])
        

In [84]:
print(pose.shape,right_hand.shape,left_hand.shape,face.shape)

(132,) (63,) (63,) (1404,)


In [6]:
extract_landmarks(landmarks).shape

(1662,)

### collecting data

<h4>Making folders</h4>

In [7]:
import os

In [17]:
data=os.path.join("Data")
no_frames=30
no_video=30
actions=["thanks","thanks"]


In [9]:
for action in actions:
    for vid in range(no_video):
        try:
            os.makedirs(os.path.join(data,action,str(vid)))
        except:
            pass

<h4>collecting frames</h4>

In [18]:
cam=cv2.VideoCapture(0)
flag=False
with mp_hol.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hol:
    for action in actions:
        if flag:
                break
        for vid in range(no_video):
            if flag:
                break
            for frame_no in range(no_frames):
                
            
                ret,frame=cam.read()
                if not ret:
                    break
                image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                landmarks=hol.process(image)
                   
                #drawing on image
                draw(frame,landmarks,mp_draw,mp_hol)
                frame=cv2.flip(frame,1)
                
                if frame_no==0:
                    cv2.putText(frame,"Starting collection",(0,50),
                                cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),4,cv2.LINE_AA)
                    cv2.putText(frame,"collecting frame for {} video no {} Frame no {}".format(action,vid,frame_no),(120,20),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow("capture",frame)
                    if cv2.waitKey(50000) or 0xFF == ord('c'): #change capture technique
                        continue
                else:
                    cv2.putText(frame,"collecting frame for {} video no {} Frame no {}".format(action,vid,frame_no),(120,20),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)

                
                feature=extract_landmarks(landmarks)
                path=os.path.join(data,action,str(vid),str(frame_no))
                np.save(path,feature)

                
                    
                cv2.imshow("capture",frame)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    flag=True
                    break
        
    cam.release()
    cv2.destroyAllWindows()


In [15]:
cam.release()
cv2.destroyAllWindows()

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
